In [1]:
import pandas as pd
import json
from ast import literal_eval
import ast
import numpy as np

delhi_cars_df=pd.read_excel("../Car Dekho/Dataset/delhi_cars.xlsx")
delhi_cars_df['city']='Delhi'

bangalore_cars_df=pd.read_excel("../Car Dekho/Dataset/bangalore_cars.xlsx")
bangalore_cars_df['city']='Banglore'

chennai_cars_df=pd.read_excel("../Car Dekho/Dataset/chennai_cars.xlsx")
chennai_cars_df['city']='Chennai'

hyderabad_cars_df=pd.read_excel("../Car Dekho/Dataset/hyderabad_cars.xlsx")
hyderabad_cars_df['city']='Hyderabad'

jaipur_cars_df=pd.read_excel("../Car Dekho/Dataset/jaipur_cars.xlsx")
jaipur_cars_df['city']="Jaipur"

kolkata_cars_df=pd.read_excel("../Car Dekho/Dataset/kolkata_cars.xlsx")
kolkata_cars_df['city']="Kolkata"


cars_df=pd.concat([delhi_cars_df,bangalore_cars_df,chennai_cars_df,hyderabad_cars_df,jaipur_cars_df,kolkata_cars_df],axis=0)
cars_df.to_csv('cars_details.csv')

In [5]:
import pandas as pd
import json
from ast import literal_eval
import ast
import numpy as np

def flatten_json(cell_data):
    if pd.isnull(cell_data):
        return {}
    try:
        # Convert the string to a list of dictionaries (handling single quotes)
        json_data = ast.literal_eval(cell_data)
        if('key' in cell_data):
            flattened_dict = {item['key']: item['value'] for item in json_data}
        else:
            flattened_dict = {item['value']: 1 for item in json_data}
        return flattened_dict
    except Exception as e:
        #print(f"Error processing data: {e}")
        return {}

def json_columns(df):
    df1=pd.DataFrame(df['new_car_detail'].map(ast.literal_eval))
    df1=pd.json_normalize(df1['new_car_detail'])
    df1['km']=pd.to_numeric(df1['km'].str.replace(",",''))
    df1=df1.drop(['owner','priceFixedText','trendingText.imgUrl','trendingText.heading','trendingText.desc','priceSaving','priceActual'],axis=1)
    df1['price']=df1['price'].str.replace(',','').str.replace('.','').str.replace('Lakh','00000').str.replace('Crore','0000000').str.replace('₹','').str.replace(" ",'')
    df1['price']=pd.to_numeric(df1['price'])
    df1

    df2=pd.DataFrame(df['new_car_overview'].map(ast.literal_eval))
    df2=pd.json_normalize(df2['new_car_overview'])
    df2.to_csv('new_car_overview.csv')
    df3=pd.read_csv('new_car_overview.csv')
    df3['top'] = df3['top'].apply(flatten_json)
    json_df = pd.json_normalize(df3['top'])
    new_car_overview=df3.drop('top', axis=1).join(json_df)
    year=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
    for months in year:
        new_car_overview['Registration Year']=new_car_overview['Registration Year'].str.replace(months,"")
    new_car_overview['Registration Year']=pd.to_numeric(new_car_overview['Registration Year'])
    new_car_overview['Seats']=pd.to_numeric(new_car_overview['Seats'].str.replace('Seats',''))
    new_car_overview['Kms Driven']=pd.to_numeric(new_car_overview['Kms Driven'].str.replace("Kms",'').str.replace(",",''))
    new_car_overview['Engine Displacement']=pd.to_numeric(new_car_overview['Engine Displacement'].str.replace("cc",''))
    new_car_overview=new_car_overview.drop(['Unnamed: 0','heading','bottomData'],axis=1)
    new_car_overview


    df4=pd.DataFrame(df['new_car_feature'].map(ast.literal_eval))
    df4=pd.json_normalize(df4['new_car_feature'])
    df4.to_csv('new_car_feature.csv')
    df5=pd.read_csv('new_car_feature.csv')
    df5['top'] = df5['top'].apply(flatten_json)
    json_df1 = pd.json_normalize(df5['top'])
    new_car_feature=df5.drop('top', axis=1).join(json_df1)
    df6=pd.json_normalize(df4['data'])
    n=len(df6.columns)
    for i in range(n):
        dfx=pd.json_normalize(df6[i])
        df6=pd.concat([df6,dfx],axis=1)
    df6=df6.drop([0,1,2],axis=1)
    df6.to_csv('new_car_features_data.csv')
    df6=pd.read_csv('new_car_features_data.csv')
    df6['list']=df6['list'].apply(flatten_json)
    json_dfx=pd.json_normalize(df6['list'])
    new_car_feature=pd.concat([new_car_feature,json_dfx],axis=1)
    new_car_feature=new_car_feature.fillna(0)
    new_car_feature=new_car_feature.drop(['Unnamed: 0','heading','data','commonIcon'],axis=1)
    new_car_feature
    
    df7=pd.DataFrame(df['new_car_specs'].map(ast.literal_eval))
    df7=pd.json_normalize(df7['new_car_specs'])
    df7.to_csv('new_car_specs.csv')
    df8=pd.read_csv('new_car_specs.csv')
    df8['top'] = df8['top'].apply(flatten_json)
    json_df2 = pd.json_normalize(df8['top'])
    new_car_specs=df8.drop('top',axis=1).join(json_df2)
    new_car_specs=new_car_specs.drop(['heading'], axis=1)
    new_car_specs['Mileage']=pd.to_numeric(new_car_specs['Mileage'].str.replace(" kmpl","").str.replace(" km/kg",""))
    new_car_specs['Engine']=pd.to_numeric(new_car_specs['Engine'].str.replace(" CC",""))
    new_car_specs['Max Power']=new_car_specs['Max Power'].str.replace("bhp","").str.replace(" (11.2) ","").str.replace(" at 4800 rpm","").str.replace("PS","").str.replace("-6100","").str.replace("Bhp","").str.replace("(66) ","")
    new_car_specs['Max Power']=new_car_specs['Max Power'].str.replace(" at 6,600 rpm","").str.replace("  at 5250 rpm","").str.replace("  at 5200 rpm","").str.replace(" at 4000rpm","")
    l=['hp','  at 5500 RPM',' at 3800 rpm','  at 4000 rpm','rpm',' kW ','-4200rpm',' HP at 3200 rpm',' BHP','Ps','ps ','(90) / 4000',' (86.7kw)',' [224] at 3800','110(150)/5700','  at 5400 rpm','  at 4000  rpm ','  at 4000 rpm ','-4200','HP at 3200','at 5400','at 4000']
    for i in l:
        new_car_specs['Max Power']=new_car_specs['Max Power'].str.replace(i,"")
    new_car_specs['Max Power']=pd.to_numeric(new_car_specs['Max Power'])
    new_car_specs['Torque']=new_car_specs['Torque'].str.replace("Nm","").str.replace("nm","").str.replace("NM","").str.replace(" (11.2) ","").str.replace(" at 4800 rpm","").str.replace("  at 3800 rpm","").str.replace(" kgm at 4,000 rpm","").str.replace(" kgm at 1750-2750rpm","")
    l1=['kgm','KGM','at 3000 RPM','at 1800-2200 rpm','at 1600-2800 rpm','at 1440-1500rpm','/ 1900','/1750-4600','at 3750 rpm','at 1750  rpm ','at 1750 rpm']
    for i in l1:
        new_car_specs['Torque']=new_car_specs['Torque'].str.replace(i,"")
    new_car_specs['Torque']=pd.to_numeric(new_car_specs['Torque'])
    new_car_specs['Wheel Size']=pd.to_numeric(new_car_specs['Wheel Size'].str.replace("R",""))
    df9=pd.json_normalize(df7['data'])
    n=len(df9.columns)
    for i in range(n):
        dfx=pd.json_normalize(df9[i])
        df9=pd.concat([df9,dfx],axis=1)
    df9=df9.drop([0,1,2],axis=1)
    df9.to_csv('new_car_specs_data.csv')
    df9=pd.read_csv('new_car_specs_data.csv')
    df9['list']=df9['list'].apply(flatten_json)
    json_df3=pd.json_normalize(df9['list'])
    l=['@4000rpm','@6400rpm','@5500-6100rpm','@6000rpm','@5400rpm','@3850rpm','@5700rpm','@5200rpm','@6200rpm','@5000-5500rpm','@5500rpm','@3750rpm','@5000rpm','@3400rpm','@5600rpm','@3600rpm','@3750-4200rpm','@3400-4600rpm','@3900rpm','@6600rpm','@4200rpm','@6250rpm','@5250rpm','@3800-4200rpm','@5678rpm','@6500rpm','@3800rpm','@3600-4400rpm','@6300rpm','@3500-4000rpm','@3750-4000rpm','@5100-6200rpm','5500-6100','@5200-6400rpm','@4000-4500rpm','@5000-6000rpm','@3800-6200rpm','@5000-6500rpm','@4250rpm','@5800-6100rpm','@2910-4500rpm','@3000rpm','@5800rpm','@3800-4400rpm','@5500-6500rpm','@4200-6000rpm','@5250-5500rpm','@3500rpm',' 6000 rpm',' 6000rpm','5200-6700rpm','6,200 (PS','5300rpm','5250-6000rpm','4000',' 5,400 (PS','6200',' 5,550 (PS','3000-3400rpm','4500-6000rpm','5300rpm','5,700 (PS','4200-6000','5500-6000rpm','4000','4000-5000rpm','4000pm','5800-6800rpm','6000 (PS','3200rpm','6000 (PS','3000-4200rpm','5200-5250rpm','4500-6200rpm','6200','5800-6800rpm',' 5,600 (PS','5850rpm','3000-4200',' 6000rpm','3200-4400rpm','5,600 (PS',' 6000rpm','6000 (PS','1620-4000rpm','4000-5000rpm','5200pm','5,600 (PS','5,600 (PS',
    ' 5,600 (PS','3200rpm','PS @','@@rpm)','@-5000rpm','@pm','@4500-rpm','@1620-rpm','(66) @','@','PS at 6,600 rpm','Bhp',' PS at 5250 rpm',' PS at 5200 rpm','PS at rpm'
    'rpm','(PS)','4,000 (PSrpm)','3600 rpm','5500 rpm ','5,500 (PSrpm)','5,250 (PSrpm)','6,500 (PSrpm)','ÃƒÂ¢??4200rpm','5,200 (PSrpm)',' (PSrpm)','5000 rpm','6500 rpm','PS at 5500 RPM','6600 rpm','at 3800 rpm','5600 rpm','4,500Ã¢â‚¬â€œrpm)','+/-250rpm','at  rpm','3,700 (PSrpm)','-6450rpm','kW   rpm / 106  40','-6000rpm','-4200rpm','6800 rpm','HP at 3200 rpm','3,900 (PSrpm)','Ps5500+/-250rpm','-rpm',',500 (PSrpm)','6,000rpm',',000 (PSrpm)','4500 rpm ','5800  rpm','(86.7kw)4,000rpm','PS  rpm ','5,800 (PSrpm)','-6400rpm','3,800 (PSrpm)','2000 rpm','4200 rpm ','-6500rpm','â€“6100rpm','at 5400 rpm','PS at   rpm ','500 (PSrpm)','6,300 (PSrpm)','PSrpm)','6800rpm','4200 rpm','rpm)','PS at  rpm '
    ,'PS at rpm','-6100','-6000pm','ÃƒÂ¢??4200rpm','hp','(','4,500Ã¢â‚¬â€œ','3,700','BHP','3,900','Ps5500','ps  ','(90) / ',',000','-6400','[224] at 3800','5,800 ','PS  ','3,800','(150)/5700','â€“6100rpm','pm','6,300','-','-5250mm','rpm'
    ,'ÃƒÂ¢??4200r','4,500Ã¢â‚¬â€œ','PS','  5,800',')/',') /',') /','â€“6100r','mm','r','Ã¢â‚¬â€œ','Ã¢??4200','4,500â€“','–6100',',',' 5500']
    json_df3['Max Power']=json_df3['Max Power'].str.replace("bhp","")
    l1=['-','Nm','nm','NM','@','1500-2750rpm','4850rpm','1650-4000rpm','4400rpm','4200rpm','4000rpm','3000-4300rpm','4500rpm','1750rpm','1750-2500rpm','1000rpm','1500-2400rpm','1750-2750rpm','1700-2700rpm','1500-2700rpm','3500rpm','1750-4500rpm','1250-3000rpm','1380-5000rpm','1450-4800rpm','1750-2800rpm','1900-2750rpm','1600-3600rpm','3300rpm','1200-3400rpm','1500-3200rpm','3000rpm','1800-2800rpm','2000rpm','1500-2800rpm','1600-2800rpm','1400-2800rpm','4500 rpm','1200-3200rpm','2250rpm','4250rpm','4600rpm','1500-3000rpm','1600-1800rpm','1750-3000rpm',' 4500 rpm','4386rpm','1200-1400rpm','1750-3250rpm','4800rpm','3750rpm','2000-2500rpm','1800-2400rpm','1500-4100rpm','1600-3000rpm','1500-4000rpm','1750-4000rpm','3100rpm','1600-2400rpm','1600rpm',' 4100rpm','1200-4000rpm','3800rpm','1750-2250rpm','1800-3000rpm','1500â€“2500rpm','1250-5000rpm','1600-4500rpm','1400-3200rpm','1370-4500rpm','1550-4400rpm','4300rpm','1400-3250rpm','1250-4600rpm','1400rpm','1600-4000rpm','1400-3700rpm','1400-3400rpm','1750-2500','3000-4000rpm','1750-2750pm','2500â€“3500rpm','1370-4500','1500-5200rpm','1500â€“4180rpm','rpm',
    'kgm','KGM','Kgm','KGm','kGM','at','(',')',' ','11.2','2000','3000','1,750','/','1900','1250','35005000','17502500','3,750','3,500','4800','35,004,500','17,503,000','4000RPM','2,500','4,250','4,000','27,504,250','2,700','1,500Ã¢â‚¬â€œ4,500','3,400','18,002,800','15,003,000','2,000','2,750','20,002,680','1,900','6,500','1,800','4,500','3,000','16002400','4,800','20002750','19,002,750','17,002,200','3,200','4,700','4,300','2500','','3800','17502750','14003400','3200','4600','18002200','16002800','4700','14401500','17503000','4200','2400','3750','1750','2200'
    ,'pm','â€“4180','+100',',950','bhp',',250','RPM','+500',',500',',7002,200',',200',',500Ã¢â‚¬â€œ','â€“','Ã¢â‚¬â€œ','–4180','–',',']
    for i in l: 
        json_df3['Max Power']=json_df3['Max Power'].str.replace(i,"")
    for i in l1:
        json_df3['Max Torque']=json_df3['Max Torque'].str.replace(i,"")
    l2=['Displacement','Max Power','Max Torque','No of Cylinder','Values per Cylinder']
    for i in l2:
        json_df3[i]=pd.to_numeric(json_df3[i])
    json_df3[['Displacement','Max Power','Max Torque','No of Cylinder','Values per Cylinder']]

    new_car_specs=pd.concat([new_car_specs,json_df3],axis=1)

    

    cars_cols_df=pd.concat([df1,new_car_overview,new_car_specs,new_car_feature],axis=1)
    cars_cols_df['city']=df['city']
    return cars_cols_df

cars_df=pd.read_csv("cars_details.csv")
df=json_columns(cars_df)
df 
df.to_csv('cols_cars_details.csv')



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression

# Load the data
data = pd.read_csv('cols_cars_details.csv')  # Replace with your actual dataset path
data=data.drop(['Unnamed: 0','data','commonIcon','Displacement','Max Power','Max Torque'],axis=1)
# Define target and features
target = 'price'
features = data.columns[data.columns != target].tolist()  # All columns except 'price'

# Function to remove outliers using IQR
def remove_outliers(df, features):
    for feature in features:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        # Define bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Filter out outliers
        df = df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]
    return df


# Remove outliers from the dataset
numerical_cols = data[features].select_dtypes(exclude=['object']).columns.tolist()
data_cleaned = remove_outliers(data, numerical_cols)

# Identify categorical and numerical columns
categorical_cols = data[features].select_dtypes(include=['object']).columns.tolist()
numerical_cols = data[features].select_dtypes(exclude=['object']).columns.tolist()

# Handle missing values and encode categorical features
# Numerical preprocessing
num_imputer = SimpleImputer(strategy='mean')
num_scaler = StandardScaler()

# Categorical preprocessing
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_encoder=OneHotEncoder(sparse_output=False,drop='first',handle_unknown='ignore')

# Preprocess numerical data
def preprocess_numerical(X):
    X_num = X[numerical_cols].copy()
    X_num_imputed = num_imputer.fit_transform(X_num)
    X_num_scaled = num_scaler.fit_transform(X_num_imputed)
    return pd.DataFrame(X_num_scaled, columns=numerical_cols)

# Preprocess categorical data
def preprocess_categorical(X):
    X_cat = X[categorical_cols].copy()
    X_cat_imputed = cat_imputer.fit_transform(X_cat)
    X_cat_encoded = pd.DataFrame(cat_encoder.fit_transform(X_cat_imputed),columns=cat_encoder.get_feature_names_out(categorical_cols))
    return X_cat_encoded

# Preprocess the entire dataset
def preprocess_data(X):
    X_num_processed = preprocess_numerical(X)
    X_cat_processed = preprocess_categorical(X)
    return pd.concat([X_num_processed, X_cat_processed], axis=1)


# Split the data
X = data[features]
X=preprocess_data(X)

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X = X.drop(to_drop, axis=1)

y = data[target]

df1=X.copy()
df1['price']=y
df1.to_csv("oneHotEncoded_df.csv")


C:\Users\mypc\AppData\Local\Temp\ipykernel_16900\2956878101.py:20: DtypeWarning: Columns (46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cols_cars_details.csv')  # Replace with your actual dataset path


In [12]:
from lazypredict.Supervised import LazyRegressor

# Initialize LazyRegressor
lazy_regressor = LazyRegressor()

# Fit and evaluate models
models = lazy_regressor.fit(X_train_selected, X_test_selected, y_train, y_test)

# Display the results
print(models)

100%|██████████| 42/42 [00:46<00:00,  1.10s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 592
[LightGBM] [Info] Number of data points in the train set: 6695, number of used features: 10
[LightGBM] [Info] Start training from score 71936369.225392
(                               Adjusted R-Squared  R-Squared           RMSE  \
Model                                                                         
LinearRegression                             0.70       0.70    71116022.25   
TransformedTargetRegressor                   0.70       0.70    71116022.25   
Lars                                         0.70       0.70    71116022.25   
LassoLars                                    0.70       0.70    71116022.26   
Lasso                                        0.70       0.70    71116022.27   
LassoCV                            

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
import pickle

# Load the data
data = pd.read_csv("oneHotEncoded_df.csv")

# Split the data
X = data.drop(['price'], axis=1)
y = data['price']  # Use Series instead of DataFrame

# Scale features
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# Feature selection
selector = SelectKBest(score_func=f_regression, k=11)
X_train_selected = selector.fit_transform(X_train, y_train)
selected_indices = selector.get_support(indices=True)

X_train_selected = pd.DataFrame(X_train_selected, columns=X.columns[selected_indices])
X_train_selected['km']=X_train['km']
X_train_selected['km']=X_train_selected['km'].fillna(np.mean(X_train_selected['km']))
X_test_selected = pd.DataFrame(selector.transform(X_test), columns=X.columns[selected_indices])
X_test_selected['km']=X_test['km']
X_test_selected['km']=X_test_selected['km'].fillna(np.mean(X_test_selected['km']))


# Initialize the Poisson Regressor
linear_regressor = LinearRegression()

# Fit the model
linear_regressor.fit(X_train_selected, y_train)

# Get the best model
best_model = linear_regressor

# Make predictions with the best model
y_pred_best = best_model.predict(X_test_selected)

# Evaluate the best model
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

#print(f"Best Parameters: {random_search.best_params_}")
print(f"Mean Squared Error (Best Model): {mse_best}")
print(f'R2 Score (Best Model): {r2_best:.2f}')
print(X_train_selected.columns)

with open('best_prediction_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)

print("Model saved as 'best_prediction_model.pkl'")

Mean Squared Error (Best Model): 5057488620536431.0
R2 Score (Best Model): 0.70
Index(['Engine Displacement', 'Torque', 'Wheel Size', 'No of Cylinder',
       'Cruise Control.1', 'Engine Start Stop Button',
       'Steering Wheel Gearshift Paddles', 'transmission_Manual',
       'oem_Mercedes-Benz', 'model_Mercedes-Benz AMG G 63',
       'model_Toyota Land Cruiser 300', 'km'],
      dtype='object')
Model saved as 'best_prediction_model.pkl'


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression

# Load the data
data = pd.read_csv('cols_cars_details.csv')  # Replace with your actual dataset path
data=data.drop(['Unnamed: 0','data','commonIcon','Displacement','Max Power','Max Torque'],axis=1)
# Define target and features
target = 'price'
features = data.columns[data.columns != target].tolist()  # All columns except 'price'

# Function to remove outliers using IQR
def remove_outliers(df, features):
    for feature in features:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        # Define bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Filter out outliers
        df = df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]
    return df


# Remove outliers from the dataset
numerical_cols = data[features].select_dtypes(exclude=['object']).columns.tolist()
data_cleaned = remove_outliers(data, numerical_cols)

# Identify categorical and numerical columns
categorical_cols = data[features].select_dtypes(include=['object']).columns.tolist()
numerical_cols = data[features].select_dtypes(exclude=['object']).columns.tolist()

# Handle missing values and encode categorical features
# Numerical preprocessing
num_imputer = SimpleImputer(strategy='mean')
#num_scaler = StandardScaler()

# Categorical preprocessing
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_encoder=OneHotEncoder(sparse_output=False,drop='first',handle_unknown='ignore')

# Preprocess numerical data
def preprocess_numerical(X):
    X_num = X[numerical_cols].copy()
    X_num_imputed = num_imputer.fit_transform(X_num)
    #X_num_scaled = num_scaler.fit_transform(X_num_imputed)
    return pd.DataFrame(X_num_imputed, columns=numerical_cols)

# Preprocess categorical data
def preprocess_categorical(X):
    X_cat = X[categorical_cols].copy()
    X_cat_imputed = cat_imputer.fit_transform(X_cat)
    X_cat_encoded = pd.DataFrame(cat_encoder.fit_transform(X_cat_imputed),columns=cat_encoder.get_feature_names_out(categorical_cols))
    return X_cat_encoded

# Preprocess the entire dataset
def preprocess_data(X):
    X_num_processed = preprocess_numerical(X)
    X_cat_processed = preprocess_categorical(X)
    return pd.concat([X_num_processed, X_cat_processed], axis=1)


# Split the data
X = data[features]
X=preprocess_data(X)

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X = X.drop(to_drop, axis=1)

y = data[target]

df1=X.copy()
df1['price']=y
df1.to_csv("oneHotEncoded_dataset.csv")


C:\Users\mypc\AppData\Local\Temp\ipykernel_8224\3770512847.py:20: DtypeWarning: Columns (46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cols_cars_details.csv')  # Replace with your actual dataset path


In [12]:
selected_data=df1[['Engine Displacement', 'Torque', 'Wheel Size', 'No of Cylinder',
       'Cruise Control.1', 'Engine Start Stop Button',
       'Steering Wheel Gearshift Paddles', 'transmission_Manual',
       'oem_Mercedes-Benz', 'model_Mercedes-Benz AMG G 63',
       'model_Toyota Land Cruiser 300', 'km']]
selected_data['price']=df1['price']
selected_data.to_csv("selected_data.csv")

C:\Users\mypc\AppData\Local\Temp\ipykernel_8224\1015734826.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['price']=df1['price']


In [28]:
ind=['Mean','Standard Deviation']
cols=['Engine Displacement', 'Torque', 'Wheel Size', 'No of Cylinder',
       'Cruise Control.1', 'Engine Start Stop Button',
       'Steering Wheel Gearshift Paddles', 'transmission_Manual',
       'oem_Mercedes-Benz', 'model_Mercedes-Benz AMG G 63',
       'model_Toyota Land Cruiser 300', 'km']

df=pd.DataFrame(index=ind,columns=cols)
for i in cols:
    df[i]['Mean']=np.mean(selected_data[i])
    df[i]['Standard Deviation']=np.std(selected_data[i])

df.to_csv("StandardScalerMapping.csv")


In [22]:
import pandas as pd

engine_displacement = int(input('engine displacement: '))
Torque = int(input('torque: '))
wheel_size = float(input('wheel size: '))
cylinder = int(input('no of cylinders: '))
cruise = int(input('Cruise Control: '))
ignitor = int(input('Start-Stop button: '))
steer = int(input('Steering Wheel Gearshift Paddles: '))
transmission = input('transmission: ')
oem = input('oem: ')
model = input('model: ')
kilometer = float(input('km: '))  # Make sure to read this as a float

# Define the columns
cols = ['Engine Displacement', 'Torque', 'Wheel Size', 'No of Cylinder',
        'Cruise Control.1', 'Engine Start Stop Button',
        'Steering Wheel Gearshift Paddles', 'transmission_Manual',
        'oem_Mercedes-Benz', 'model_Mercedes-Benz AMG G 63',
        'model_Toyota Land Cruiser 300', 'km']

# Create a DataFrame with one row
X = pd.DataFrame([[engine_displacement, Torque, wheel_size, cylinder,
                   cruise, ignitor, steer, 
                   1 if transmission == 'Manual' else 0,
                   1 if oem == 'Mercedes-Benz' else 0,
                   1 if model == 'Mercedes-Benz AMG G 63' else 0,
                   1 if model == 'Toyota Land Cruiser 300' else 0,
                   kilometer]], columns=cols)

X

,Engine Displacement,Torque,Wheel Size,No of Cylinder,Cruise Control.1,Engine Start Stop Button,Steering Wheel Gearshift Paddles,transmission_Manual,oem_Mercedes-Benz,model_Mercedes-Benz AMG G 63,model_Toyota Land Cruiser 300,km
0,1493,250,17.0,4,1,1,1,0,0,0,0,10000.0


In [23]:
df=pd.read_csv('StandardScalerMapping.csv')
for i in df.columns[1:]:
    a=df[i]
    X[i]=(X[i]-a[0])/a[1]
X

,Engine Displacement,Torque,Wheel Size,No of Cylinder,Cruise Control.1,Engine Start Stop Button,Steering Wheel Gearshift Paddles,transmission_Manual,oem_Mercedes-Benz,model_Mercedes-Benz AMG G 63,model_Toyota Land Cruiser 300,km
0,0.142965,0.659392,1.068446,0.332835,1.497512,1.383473,3.130289,-1.616168,-0.173659,-0.015461,-0.010932,-0.661309


In [24]:
import pickle
with open('best_prediction_model.pkl', 'rb') as file:
    model = pickle.load(file)

model.predict(X)

array([75430164.22706454])